In [ ]:
%pip install --upgrade -r requirements.txt

Create the Pay-i and Bedrock clients

In [ ]:
import boto3
import json
import os
import urllib3

from payi import Payi
from payi.lib.instrument import payi_instrument, ingest

# Read the API KEYs from the environment, replace the default values (the second argument) with your own keys if needed
payi_api_key = os.getenv("PAYI_API_KEY", "YOUR_PAYI_API_KEY")

# Substitute the region for your regional deployment
region_name = "us-west-2"

payi = Payi(
    api_key=payi_api_key,
)
 
# Instrument the bedrock client so that the @ingest decorator can be used
payi_instrument(payi)

bedrock = boto3.client(
    'bedrock-runtime',
    region_name=region_name,
    )

Create an experience type and limit.  Both will be used when ingesting data into Pay-i

In [ ]:
experience_name = 'bedrock_ingest_experience_test'
payi.experiences.types.create(name=experience_name, description='This is a test experience')
 
result = payi.limits.create(max=10_000, limit_name='bedrock_ingest_limit')
limit_id = result.limit.limit_id

print(f"Created limit with id {limit_id}")

Decorate a method which calls the bedrock client

In [ ]:
from payi.lib.helpers import create_headers

# The typical decorator usage pattern is to specify request tags and/or experience name as they the same values will be applied to all bedrock calls in the decorated function,
# while user_id and limits are typically specified in each bedrock call to attribute usage to a user and/or limit

# When experience_name is provided and experience_id is not, the experience instance id will be automatically created and the same ID will be applied to all bedrock calls
@ingest(request_tags=['x', 'y'],  experience_name=experience_name)
def call_bedrock(client, limit_id, user_id):
    
    request_dict = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": "this is a test"}],
            }
        ],
    }

    # Convert the request to JSON
    request_body = json.dumps(request_dict)
    model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

    # Invoke the model with the request.
    invoke_response = bedrock.invoke_model(
        modelId=model_id,
        body=request_body,
        # To pass per method metadata to ingest, specify each metadata value in the call to create_headers.  
        # These additional header values will not be sent to bedrock.
        extra_headers=create_headers(user_id=user_id, limit_ids=[limit_id])
    )
    
    # Decode the response body.
    response = invoke_response["body"].read()

    response_json = json.loads(response)
    print(json.dumps(response_json, indent=4))

    # Additional calls to bedrock (invoke_model_with_response_stream, converse, converse_stream) can be made here and will be ingested.
    # If another method is called, it's calls to bedrock will also be ingested.
    ...


Call the decorated method to call bedrock and ingest the results through the Pay-i instrumentation

In [ ]:
call_bedrock(bedrock, limit_id, 'a_user_id')

See that the limit input, output, and total base cost has been updated with the ingest data

In [ ]:
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.output.base}")

Reset a limit back to zero tracked cost

In [ ]:
print("\nState prior to reset:")
response = payi.limits.reset(limit_id=limit_id)
print(f"Limit Name: {response.limit_history.limit_name}")
print(f"Limit ID: {response.limit_history.limit_id}")
print(f"Limit Reset Timestamp: {response.limit_history.limit_reset_timestamp}")
print(f"Limit Tags: {response.limit_history.limit_tags}")
print(f"Limit Input Base Cost: {response.limit_history.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit_history.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit_history.totals.cost.total.base}")

print("\nState after reset:")
response = payi.limits.retrieve(limit_id=limit_id)
print(f"Limit Name: {response.limit.limit_name}")
print(f"Limit ID: {response.limit.limit_id}")
print(f"Limit Creation Timestamp: {response.limit.limit_creation_timestamp}")
print(f"Limit Tags: {response.limit.limit_tags}")
print(f"Limit Input Base Cost: {response.limit.totals.cost.input.base}")
print(f"Limit Output Base Cost: {response.limit.totals.cost.output.base}")
print(f"Limit Total Base Cost: {response.limit.totals.cost.total.base}")